# 理解 Kernel和配置

## 配置AI服务
Kernel 是Semantic Kernel中的总调度员.

当开始一个SK项目时，我们首先创建 Kernel 实例. 根据需要，我们可以有多个Kernel 实例并配置不同的 AI 服务。

SK沿用我们常见的Builder范式来配置 Logger，AI 服务， Plugin, 提示词模板并创建 Kernel 对象。

(注：Semantic Kernel目前的dotnet版本仅支持OPENAI 和 Azure OPENAI) 

In [4]:
// 安装 nuget 包
#r "nuget: Microsoft.SemanticKernel, 1.0.0-beta8"
#r "nuget: Microsoft.Extensions.Logging.Console, 6.0.0.0"

Installed Packages Microsoft.Extensions.Logging.Console, 6.0.0 Microsoft.SemanticKernel, 1.0.0-beta8

In [5]:
using Microsoft.Extensions.Logging;
using Microsoft.SemanticKernel;

//Create Kernel builder
var builder = new KernelBuilder();

//此lab中我们使用OPENAI服务. 如使用Azure OPENAI，请取消以下注释
/*
var model = "gpt-35-turbo";
var apiKey = "<your apikey>";
azureEndpoint="<your azure openai endpoint>";
ArgumentNullException.ThrowIfNull(apiKey);
ArgumentNullException.ThrowIfNull(azureEndpoint);
builder.WithAzureOpenAIChatCompletionService(model, azureEndpoint, apiKey);
*/
var model = "gpt-3.5-turbo-1106";
var apiKey = Environment.GetEnvironmentVariable("OPENAI__APIKEY");

ArgumentNullException.ThrowIfNull(apiKey);

var loggerFactory = LoggerFactory.Create(builder => { 
    builder.SetMinimumLevel(LogLevel.Information);  
    builder.AddConsole(); });

var kernel = builder
.WithLoggerFactory(loggerFactory)
.WithOpenAIChatCompletionService(model, apiKey)
.Build();

我们配置了一个极简的Kernel。现在用它来调用OPEN AI的服务

In [8]:
const string JokeFuncDefinition = "请讲个关于{{ $input}} 的笑话, 要求用黑色幽默";

var jokeFunc = kernel.CreateSemanticFunction(JokeFuncDefinition);

var result = await kernel.RunAsync("Canada", jokeFunc);
Console.WriteLine(result.GetValue<string>());

为什么加拿大人总是那么冷静？因为他们知道，无论多冷，总有一天夏天会来。


以上的例子中Kernel做了几件事：我们创建了一个生成笑话的 Semanitc Function。

让我们仔细探询一下返回的结果，留心 PluginName 和 RenderedPrompt。

kernel在此作为runtime 运行该 Semantic Function。

首先它使用默认的Prompt Engine 将`jokeFunc` 中的Prompt Template 与输入变量一起生成 Prompt 文本，然后调用 OPENAI （也是一个Plugin）的服务生成结果。

In [10]:
result.FunctionResults.First()


为什么加拿大人总是那么冷静？因为他们知道，无论多冷，总有一天夏天会来。 FunctionName func89404a2240f2492a9016bc135a7eaf57 PluginName _GLOBAL_FUNCTIONS_ Metadata key type value ModelResults Microsoft.SemanticKernel.Orchestration.ModelResult[] index value 0 Microsoft.SemanticKernel.Orchestration.ModelResult RenderedPrompt System.String 请讲个关于Canada 的笑话, 要求用黑色幽默

另外，Kernel还负责执行管道 (pipeline)

In [12]:
const string translateFuncDefinition = "请讲以下输入翻译成英文： {{ $input}}";

var translateToEngFunc = kernel.CreateSemanticFunction(translateFuncDefinition);
var result2 = await kernel.RunAsync("United States", jokeFunc, translateToEngFunc);
Console.WriteLine(result2.GetValue<string>());

Error: System.Threading.Tasks.TaskCanceledException: The operation was cancelled because it exceeded the configured timeout of 0:01:40. Network timeout can be adjusted in ClientOptions.Retry.NetworkTimeout.
 ---> System.Threading.Tasks.TaskCanceledException: The operation was canceled.
 ---> System.IO.IOException: Unable to read data from the transport connection: The I/O operation has been aborted because of either a thread exit or an application request..
 ---> System.Net.Sockets.SocketException (995): The I/O operation has been aborted because of either a thread exit or an application request.
   --- End of inner exception stack trace ---
   at System.Net.Sockets.Socket.AwaitableSocketAsyncEventArgs.ThrowException(SocketError error, CancellationToken cancellationToken)
   at System.Net.Sockets.Socket.AwaitableSocketAsyncEventArgs.System.Threading.Tasks.Sources.IValueTaskSource<System.Int32>.GetResult(Int16 token)
   at System.Net.Security.SslStream.EnsureFullTlsFrameAsync[TIO

## 实践： 将英文再翻译成法文输出

In [32]:
// you code here